### Libraries

In [1]:
import requests 
import json
from dotenv import load_dotenv
import os
import pandas as pd

from functools import reduce
import operator
import geopandas as gdp
import shapely


from pymongo import MongoClient

In [2]:
load_dotenv()

True

### Creando los geocodes

En este caso, voy a partir de tres ubicaciones diferentes:

- España --> Madrid
- Paises Bajos --> Amsterdam
- Francia --> Paris `latitude:48.8709209 longitude:2.3040091`
- Canadá --> Montréal --> `latitude:42.8839162 longitude:-8.5255155`

In [3]:
countries = ["madrid","paris", "ottawa"]

EN ESTA FUNCIOÓN DE AQUÍ TENDRÍA QUE MIRAR PARA METER EL TRY AND EXCEPT

In [4]:
def get_coordinates(lista):
    d = {}

    for country in countries:
        locations = requests.get(f'https://geocode.xyz/{country}?json=1')
        locations = locations.json()
        d[country] = locations
        
    return d     

In [5]:
c = get_coordinates(countries)

In [6]:
c['madrid']
for k, v in c.items():
    print(v.keys())

dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])
dict_keys(['standard', 'longt', 'alt', 'elevation', 'latt'])


In [7]:
def exctract_coordinates(dictionary):
    dict_empty = {}
    for k, v in c.items():
        coor = [float(v["longt"]),float(v["latt"])]
        dict_empty[k] = coor
    return dict_empty

In [8]:
coordinates = exctract_coordinates(c)

In [9]:
coordinates

{'madrid': [-3.6793, 40.42955],
 'paris': [2.3428, 48.85756],
 'ottawa': [-75.70531, 45.35917]}

### Llamadas a la API

meter un parametro de radio = metros desde la ubicación dada
meter el parametro de limit para que no me pete, tiene un limite a la hora

In [10]:
token1= os.getenv("token1")
token2 = os.getenv("token2")

In [11]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

#### Llamadas a la API

#####  Codes to queries

Dentro de todas las queries hay algunas que tienen que estar muy cerca o otras que da igual que estén un pco más lejos, lo que haré separarlas por grupos

In [12]:
school = '52e81612bcbc57f1066b7a45'
vegan = "4bf58dd8d48988d1d3941735"

queries = [school, vegan]


##### Setting the parameters

In [13]:
def datos_madrid (*args):
    '''
    descripcion de lo que hace la función aquí
    '''  
    d = {}
    
    for i in args: 
        parametros = {"client_id" : token1,
                  "client_secret" : token2,
                  "v": "20180323",
                  "ll": f"{coordinates['madrid'][1]},{coordinates['madrid'][0]}", ## esto lo puedo hacer con kwargs
                  "query":i,
                  "limit": 100}  

        resp = requests.get(url= url_query, params=parametros)
        data = json.loads(resp.text)
        d[i] = data
    return d

In [14]:
results = datos_madrid(*[school, vegan])

In [15]:
results.keys()#salen como resultado 7 diccionarios de cada una de las queries que he pedido. Despues de esto habría que 
#separar cada uno de los diccionarios y proceder a su limpieza

dict_keys(['52e81612bcbc57f1066b7a45', '4bf58dd8d48988d1d3941735'])

In [16]:
len(results)

2

**Para separar los distintos diccionarios que he genarado con la función anterior:**

In [17]:
filterByKey = lambda keys: {x: results[x] for x in keys}
queries_codes = {"schoolKey":'52e81612bcbc57f1066b7a45', 
                "veganKey": "4bf58dd8d48988d1d3941735"}

print(queries_codes['schoolKey'])
print(queries_codes['veganKey'])

52e81612bcbc57f1066b7a45
4bf58dd8d48988d1d3941735


In [18]:
def create_separate_dictionaries (dictonary_queries):
    filterByKey = lambda keys: {x: results[x] for x in keys}
    for key, values in dictonary_queries.items():
        
        return filterByKey([values])

### Limpieza de los resultados


In [19]:
schoolKeys = ['52e81612bcbc57f1066b7a45']
veganKeys = ["4bf58dd8d48988d1d3941735"]

school = filterByKey(schoolKeys)
vegan = filterByKey(veganKeys)

all_info = [school, vegan]   

In [20]:
def cleaning_dictionaries(*args):
    for dictionary in args:
        for key, values in dictionary.items():
            
            response = values.get('response')
            decoded = response.get('groups')[0]
            final = decoded.get('items')
    return final

In [21]:
def cleaning_dictionaries(*args):
      for dictionary in args: 
        for key, values in dictionary.items():
            if key == '52e81612bcbc57f1066b7a45':
            
                response = values.get('response')
                decoded = response.get('groups')[0]
                final_school = decoded.get('items')
                
            elif key == "4bf58dd8d48988d1d3941735":
                
                response = values.get('response')
                decoded = response.get('groups')[0]
                final_vegan = decoded.get('items')
  
                return final_school, final_vegan

In [22]:
school_data,vegan_data= cleaning_dictionaries(*[school, vegan])

### Creating dataframes

In [23]:
mapa_nombre = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [24]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

####  For school

In [25]:
data_schools = []
for dic in school_data:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_schools.append(paralista)

In [26]:
data_schools[0]

{'name': 'Gourmet Experience',
 'latitud': 40.42777668625794,
 'longitud': -3.687295697667445}

####  For vegan

In [27]:
data_vegan = []
for dic in vegan_data:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    data_vegan.append(paralista)
data_vegan[0]

{'name': 'Vinos De Bellota',
 'latitud': 40.42067595716258,
 'longitud': -3.679058690556566}

###  Saving dataframes

#### School query

In [31]:
df_pets = pd.DataFrame(data_schools)
df_pets.to_csv("school.csv", index = False)
df_pets.head()

,name,latitud,longitud
0,Gourmet Experience,40.427777,-3.687296
1,Monumento a Alfonso XII de España,40.417081,-3.683140
2,Fox In A Box Madrid - Juego de Escape Game,40.420115,-3.697489
3,Fuencarral 52 B&B,40.423500,-3.700465
4,B&B Restaurante,40.421384,-3.697403


#### Vegan query

In [32]:
df_gym = pd.DataFrame(data_vegan)
df_gym.to_csv("vegan.csv", index = False)
df_gym.head()

,name,latitud,longitud
0,Vinos De Bellota,40.420676,-3.679059
1,De María,40.425839,-3.697331
2,D'Mystic,40.423215,-3.698125
3,SANZ CABREJAS Bufete de Abogados penalista,40.435540,-3.686364
4,D'kebab Food,40.422261,-3.699776
